In [1]:
    """
    The role of this file is to provide the functions used to 
    transmit the parsed data to the SPI.

    Some of the following things might have to be removed when going for the SPI :
    - time library 
    - len_last_wr_data

    TO CHANGE : FOR TEST PURPOSES WE RECORD THE WRITTEN DATA : IN FACT, WE GET THE DATA BACK FROM WRITTING AND NOT LOOKING AT THE MEM ...

    """
    
    # command_disjunction only gives the first level of behaviour of each command
    # returns the list of rd_data to executue the compare command --> might be changed ...
    def command_disjunction(AxiQspi, result_file_html, result_file_txt, indice_command_from_test_file : int, parsed_line : list, last_rd_data : list) -> None :
        command = parsed_line[0]
        executing_line_msg(result_file_html, result_file_txt, command, parsed_line)

        if ((command == "REGRD" )):
            spi_cmd = spi_commands["SPI_REGRD"]
            rd_addr = parsed_line[1]
            last_rd_data = rd_transaction_behaviour(AxiQspi, result_file_html, result_file_txt, command, spi_cmd, rd_addr)

        elif ((command == "REGWR" )):
            spi_cmd = spi_commands["SPI_REGWR"]
            wr_addr = parsed_line[1]
            wr_data = parsed_line[2]
            wr_transaction_behaviour(AxiQspi, result_file_html, result_file_txt, command, spi_cmd, wr_addr, wr_data)

        elif (command == "MEMRD") :
            spi_cmd = spi_commands["SPI_MEMRD"]
            rd_addr = parsed_line[1]
            last_rd_data = rd_transaction_behaviour(AxiQspi, result_file_html, result_file_txt, command, spi_cmd, rd_addr)

        elif (command == "MEMWR"):
            spi_cmd = spi_commands["SPI_MEMWR"]
            wr_addr = parsed_line[1]
            wr_data = parsed_line[2]
            wr_transaction_behaviour(AxiQspi, result_file_html, result_file_txt, command, spi_cmd, wr_addr, wr_data)

        elif (command == "MEMRDM") :
            spi_cmd = spi_commands["SPI_MEMRDM"]
            rd_addr = parsed_line[1] # <=> None here
            last_rd_data = rd_transaction_behaviour(AxiQspi, result_file_html, result_file_txt, command, spi_cmd, rd_addr)

        elif (command == "MEMWRM") :
            spi_cmd = spi_commands["SPI_MEMWRM"]
            wr_addr = parsed_line[1] # <=> None here
            wr_data = parsed_line[2] # <=> None here
            wr_transaction_behaviour(AxiQspi, result_file_html, result_file_txt, command, spi_cmd, wr_addr, wr_data)

        elif (command == "REGCMP"):
            ref_packets = parsed_line[2] 
            ref_mask = ref_packets[::nb_token_to_compare_reg_cmp]
            ref_data = ref_packets[nb_token_to_compare_reg_cmp::]
            
            if (is_empty(last_rd_data)):
                no_rd_data(result_file_html, result_file_txt)
            else :
                reg_cmp_behaviour(result_file_html, result_file_txt, command, ref_mask, ref_data, last_rd_data)
                
        elif (command == "MEMCMP"):
            data_ref = parsed_line[2] 
            if (is_empty(last_rd_data)):
                no_rd_data(result_file_html, result_file_txt)
            else :
                mem_cmp_behaviour(result_file_html, result_file_txt, command, data_ref, last_rd_data)

        else :
            unknown_cmd(result_file_html, result_file_txt, command)
        return last_rd_data



    # Implements the basic behaviour of the read commands
    # return a string composed of read bytes in the same format than the test file
    def rd_transaction_behaviour(AxiQspi, result_file_html, result_file_txt, command : str, spi_cmd : int, rd_addr : list) -> None:
        valid_rd, rd_line = rd_spi_transaction(AxiQspi, result_file_html, result_file_txt, command, spi_cmd, rd_addr)
        nb_hex_per_token_data = nb_bytes_per_token_data * 2 # because one byte = 2 hex char
        formated_rd_line = format_rd_data(rd_line, nb_hex_per_token_data)
        if valid_rd :
            valid_rd_msg(result_file_html, result_file_txt, indice_command_from_test_file, formated_rd_line)
        else :
            spi_cmd_hex = from_int_to_hex(spi_cmd)
            error_rd_msg(result_file_html, result_file_txt, spi_cmd_hex)
        return rd_line
            
    # Implements the basic behaviour of the write commands
    def wr_transaction_behaviour(AxiQspi, result_file_html, result_file_txt, command : str, spi_cmd : int, wr_addr : list, wr_data : list) -> None:
        valid_wr = wr_spi_transaction(AxiQspi, result_file_html, result_file_txt, command, spi_cmd, wr_addr, wr_data)
        if valid_wr :
            valid_wr_msg(result_file_html, result_file_txt, indice_command_from_test_file)
        else :
            spi_cmd_hex = from_int_to_hex(spi_cmd)
            error_wr_msg(result_file_html, result_file_txt, spi_cmd_hex)

    
    # Implements behaviour of the reg cmp command
    # data_ref is here composed of a mask hex packet and a written data hex packet ; hex_packet being composed of nb_hex_per_packet
    # ref_mask and ref_data MUST have the same length (which would make sense)
    
    # testing of length will be removed 
    def reg_cmp_behaviour(result_file_html, result_file_txt, command : str, ref_mask : list, ref_data : list, last_rd_data : list) -> None :
        starting_cmp(result_file_html, result_file_txt)
        storage = "reg"
        
        # Data formatting to be able to compare
        parsed_ref_mask = parse_list_of_packets(ref_mask, "int")
        parsed_ref_data = parse_list_of_packets(ref_data, "int")
        
        nb_rd_bytes = len(last_rd_data)
        len_ref_data = len(parsed_ref_data)
        result = 0
        if (nb_rd_bytes != len_ref_data): 
            # print("these are the data mismatch :", ref_data, last_rd_data)
            number_of_packets_mismatch(result_file_html, result_file_txt, len_ref_data, nb_rd_bytes)
            result += 1
        else :
            #Here from the MSB to the LSB 
            for hexa_byte_cnt in range(len_ref_data):
                if (parsed_ref_mask[hexa_byte_cnt] & parsed_ref_data[hexa_byte_cnt] != 0):
                    ref_byte = parsed_ref_data[hexa_byte_cnt] # we compare only data bytes : not mask bytes
                    rd_byte = last_rd_data[hexa_byte_cnt]
                    result += equal_bytes(result_file_html, result_file_txt, ref_byte, rd_byte, storage) # if not equal : raise Exception : see messsages.ipynb
                    
        nb_hex_per_token_data = nb_bytes_per_token_data * 2          
        formated_ref_data = format_rd_data(parsed_ref_data, nb_hex_per_token_data)
        formated_rd_data = format_rd_data(last_rd_data, nb_hex_per_token_data)
        if (result == 0):    
            valid_cmp(result_file_html, result_file_txt, formated_ref_data, formated_rd_data)
        else :
            invalid_cmp(result_file_html, result_file_txt, formated_ref_data, formated_rd_data)
            
    # Implements behaviour of the mem cmp command
    # data_ref is composed of a certain nb of 8 hex characteres packets ; hex_packet being composed of nb_hex_per_packet
    def mem_cmp_behaviour(result_file_html, result_file_txt, command : str, data_ref : list, last_rd_data : list) -> None :
        starting_cmp(result_file_html, result_file_txt)
        storage = "mem"
        nb_rd_bytes = len(last_rd_data)
        len_ref_data = len(data_ref)
        result = 0
        if (len_ref_data != nb_rd_bytes): 
            # print("these are the number of bytes mismatch :", len_ref_data, nb_rd_bytes)
            number_of_packets_mismatch(result_file_html, result_file_txt, len_ref_data, nb_rd_bytes)
            result += 1
        else :
            #Here from the MSB to the LSB 
            for hexa_packet_cnt in range(len_ref_data):
                rd_hex_packet = last_rd_data[hexa_packet_cnt]
                hex_packet_ref = data_ref[hexa_packet_cnt]
                result += equal_bytes(result_file_html, result_file_txt, hex_packet_ref, rd_hex_packet, storage)
        if (result == 0):    
            valid_cmp(result_file_html, result_file_txt, data_ref, last_rd_data)
        else :
            invalid_cmp(result_file_html, result_file_txt, data_ref, last_rd_data)
        

    # Test if two given bytes are equal and writes the result in the result file 
    def equal_bytes(result_file_html, result_file_txt, ref_byte : int, rd_byte : int, storage : str) -> int :
        # hex representation to display messages
        ref_hex_byte = from_int_to_hex(ref_byte)
        rd_hex_byte = from_int_to_hex(rd_byte)
        
        if (ref_byte != rd_byte): # in case hex data differs
            if (storage == "reg"):
                reg_cmp_mismatch(result_file_html, result_file_txt, ref_hex_byte, rd_hex_byte)
            elif (storage == "mem"):
                mem_cmp_mismatch(result_file_html, result_file_txt, ref_hex_byte, rd_hex_byte)
            return 1
        else :
            if (storage == "reg"):
                reg_cmp_match(result_file_html, result_file_txt, ref_hex_byte, rd_hex_byte)
            elif (storage == "mem"):
                mem_cmp_match(result_file_html, result_file_txt, ref_hex_byte, rd_hex_byte)
            return 0


    # get_bytes returns the list of bytes stored at a given address (addr)
    def rd_spi_transaction(AxiQspi, result_file_html, result_file_txt, command : str, spi_cmd : int, addr : list) :
        rd_line = []
        data_number_token_to_read = dict_data_number_token_to_read[command]
        
        # Send command to SPI
        xfer(AxiQspi, [spi_cmd]) # we don't get the first byte
        
        # Send address to SPI
        if (addr):
            for addr_token in addr:
                parsed_addr_token = parse_packet(addr_token,"int")
                formated_addr = format_token_field(nb_bytes_per_token_addr, parsed_addr_token)
                xfer(AxiQspi, formated_addr) 
                
        transmitted_line_int = [spi_cmd] + formated_addr
        transmitted_line_hex = from_list_int_to_hex(transmitted_line_int)
        sending_msg(result_file_html, result_file_txt, command, transmitted_line_hex)
            
        # Waiting for command 
        valid_rd = wait_for_spi_cmd(AxiQspi, spi_cmd) 

        # receive data 
        nb_bytes_to_get = data_number_token_to_read * nb_bytes_per_token_data
        data_to_send = [0x00 for i in range(nb_bytes_to_get)]
        rd_line = xfer(AxiQspi, data_to_send) 
        rd_line.reverse()
           
        return valid_rd, rd_line 



    # write_val writes the list of bytes (data) at a given address (addr)
    # the format of written_line is : [cmd_byte,[addr_bytes],[data_bytes]]
    def wr_spi_transaction(AxiQspi, result_file_html, result_file_txt, command : str, spi_cmd : int, addr : list, data : list) :        
        test_wr_data = []
        data_number_token_to_write = dict_data_number_token_to_write[command]
        
        # Send command to SPI
        xfer(AxiQspi, [spi_cmd])


        # Send address to SPI
        if (addr):
            for addr_token in addr:
                parsed_addr_token = parse_packet(addr_token,"int")
                formated_addr = format_token_field(nb_bytes_per_token_addr, parsed_addr_token)
                xfer(AxiQspi, formated_addr)   
            
        
        # Send data to SPI
        if (data):
            formated_data = []
            for indice_token in range (data_number_token_to_write) : 
                token_to_send = data[indice_token]
                parsed_data_token = parse_packet(token_to_send,"int")
                formated_token = format_token_field(nb_bytes_per_token_data, parsed_data_token)
                formated_data = formated_data + formated_token 
                xfer(AxiQspi, formated_token)
        
        transmitted_line_int = [spi_cmd] + formated_addr + formated_data
        transmitted_line_hex = from_list_int_to_hex(transmitted_line_int)
        sending_msg(result_file_html, result_file_txt, command, transmitted_line_hex)
            
        # Waiting for command 
        valid_wr = wait_for_spi_cmd(AxiQspi, spi_cmd)        
        return valid_wr
    
    
    def wait_for_spi_cmd(AxiQspi, spi_cmd : int) -> int:
        # len_cmd = len(spi_cmd)
        nb_iterations = 0
        current_byte = []
        cmd_received = 0
        # print(f"searching for command {spi_cmd}")
        while ((current_byte != spi_cmd) and (nb_iterations < max_while_iterations)) :
            current_byte_list = xfer(AxiQspi, [0x00])
            current_byte = current_byte_list[0]
            # if current_byte != 0:
            #     potential_cmd_byte = current_byte
            #     if len_cmd > 1 :
            #         data_to_send = [0x00 for i in range (len_cmd-1)]
            #         potential_cmd_byte = 
            #     cmd_hex = from_int_to_hex(current_byte)
            #     print(f"Current iteration is {nb_iterations} with byte {cmd_hex}")
            nb_iterations += 1
        if nb_iterations < max_while_iterations :
            cmd_received = 1
        else :
            cmd_received = 0
        return cmd_received